In [4]:
import pandas as pd
import numpy as np
from keras.models import Sequential, Model
from sklearn.model_selection import train_test_split
from keras.layers import LSTM
from keras.layers import Dense, Input,Flatten, Dropout, TimeDistributed, RepeatVector
from keras.optimizers import Adam
from keras import backend as K
import keras
import matplotlib.pyplot as plt
import utm
from sklearn.preprocessing import minmax_scale, MinMaxScaler
from keras.utils import to_categorical

class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = {'batch': [], 'epoch': []}
        self.accuracy = {'batch': [], 'epoch': []}
        self.val_loss = {'batch': [], 'epoch': []}
        self.val_acc = {'batch': [], 'epoch': []}

    def on_batch_end(self, batch, logs={}):
        self.losses['batch'].append(logs.get('loss'))
        self.accuracy['batch'].append(logs.get('acc'))
        self.val_loss['batch'].append(logs.get('val_loss'))
        self.val_acc['batch'].append(logs.get('val_acc'))

    def on_epoch_end(self, batch, logs={}):
        self.losses['epoch'].append(logs.get('loss'))
        self.accuracy['epoch'].append(logs.get('acc'))
        self.val_loss['epoch'].append(logs.get('val_loss'))
        self.val_acc['epoch'].append(logs.get('val_acc'))

    def loss_plot(self, loss_type):
        iters = range(len(self.losses[loss_type]))
        plt.figure()
        # acc
        plt.plot(iters, self.accuracy[loss_type], 'r', label='train acc')
        # loss
        plt.plot(iters, self.losses[loss_type], 'g', label='train loss')
        if loss_type == 'epoch':
            # val_acc
            plt.plot(iters, self.val_acc[loss_type], 'b', label='val acc')
            # val_loss
            plt.plot(iters, self.val_loss[loss_type], 'k', label='val loss')
        plt.grid(True)
        plt.xlabel(loss_type)
        plt.ylabel('acc-loss')
        plt.legend(loc="upper right")
        plt.show()


def distance_loss(y_pred, y_true):
    return K.sqrt(K.mean(K.sum(K.square(y_pred-y_true), axis=-1)))

def median_absolute_deviation(y_pred, y_true):
    deviation = np.abs(y_pred-y_true)
    return np.mean(deviation, axis=0)


data = pd.read_csv('new_formatted_data2.csv')
label = pd.read_csv('label1.csv')['label']
X_temp = [
    np.concatenate([mr[4:9],mr[9:14],mr[14:19],mr[19:24],mr[24:29],
               mr[29:34]]) for mr in data.values
]
X_temp = np.array(X_temp)
y_temp = label.values

traj = [0,1,2,3,4,5,6,7,8,10]
length = [22,120,28,36,39,22,83,300,272]
X = []
y = []
for t in range(72):
    temp = X_temp[data['TrajID'] == t]
    temp_y = y_temp[data['TrajID'] == t]
    for i in range(len(temp)-8):
        X.append(temp[i:i+8])
        y.append(temp_y[i:i+8])
X = np.array(X)
y = np.array(y)
y = to_categorical(y, 885)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=33)
history = LossHistory()

def build_model_ae():
    m = Sequential()
    m.add(LSTM(64, return_sequences=True, input_shape=(8,30)))
    m.add(LSTM(32, return_sequences=True))
    m.add(TimeDistributed(Dense(30)))
    r = Adam(lr=0.001)
    m.compile(optimizer=r, loss='mse', metrics=['mse'])
    return m

def build_model_lstm():
    m = Sequential()
    m.add(LSTM(32, return_sequences=True, input_shape=(8, 64)))
    m.add(LSTM(64, return_sequences=True))
    m.add(TimeDistributed(Dense(1024, activation='relu')))
    m.add(TimeDistributed(Dense(1024, activation='relu')))
    m.add(TimeDistributed(Dense(885, activation='softmax')))
    r = Adam(lr=0.001)
    m.compile(optimizer=r, loss='categorical_crossentropy', metrics=['accuracy'])
    return m

model1 = build_model_ae()
model1.fit(X_train, X_train, validation_data=(X_test, X_test), epochs=100, batch_size=32)
model1 = Model(inputs=model1.inputs, outputs=model1.layers[0].output)
model1.summary()
X_train_vec = model1.predict(X_train)
X_test_vec = model1.predict(X_test)
model2 = build_model_lstm()
model2.fit(X_train_vec, y_train, validation_data=(X_test_vec, y_test),epochs=100, batch_size=64)
model2.summary()
# print(model2.predict(X_test))
print(model2.evaluate(X_test_vec, y_test))

Train on 7576 samples, validate on 1895 samples
Epoch 1/100
7576/7576 [==============================] - 13s 2ms/step - loss: 0.0516 - mean_squared_error: 0.0516 - val_loss: 0.0100 - val_mean_squared_error: 0.0100
Epoch 2/100
7576/7576 [==============================] - 12s 2ms/step - loss: 0.0077 - mean_squared_error: 0.0077 - val_loss: 0.0068 - val_mean_squared_error: 0.0068
Epoch 3/100
7576/7576 [==============================] - 12s 2ms/step - loss: 0.0056 - mean_squared_error: 0.0056 - val_loss: 0.0052 - val_mean_squared_error: 0.0052
Epoch 4/100
7576/7576 [==============================] - 12s 2ms/step - loss: 0.0042 - mean_squared_error: 0.0042 - val_loss: 0.0040 - val_mean_squared_error: 0.0040
Epoch 5/100
7576/7576 [==============================] - 12s 2ms/step - loss: 0.0033 - mean_squared_error: 0.0033 - val_loss: 0.0032 - val_mean_squared_error: 0.0032
Epoch 6/100
7576/7576 [==============================] - 11s 2ms/step - loss: 0.0027 - mean_squared_error: 0.0027 - val_lo

Epoch 91/100
7576/7576 [==============================] - 12s 2ms/step - loss: 9.4936e-06 - mean_squared_error: 9.4936e-06 - val_loss: 1.8632e-05 - val_mean_squared_error: 1.8632e-05
Epoch 92/100
7576/7576 [==============================] - 12s 2ms/step - loss: 7.9639e-06 - mean_squared_error: 7.9639e-06 - val_loss: 7.4961e-06 - val_mean_squared_error: 7.4961e-06
Epoch 93/100
7576/7576 [==============================] - 12s 2ms/step - loss: 8.8368e-06 - mean_squared_error: 8.8368e-06 - val_loss: 9.4493e-06 - val_mean_squared_error: 9.4493e-06
Epoch 94/100
7576/7576 [==============================] - 12s 2ms/step - loss: 9.0844e-06 - mean_squared_error: 9.0844e-06 - val_loss: 7.5039e-06 - val_mean_squared_error: 7.5039e-06
Epoch 95/100
7576/7576 [==============================] - 12s 2ms/step - loss: 7.8168e-06 - mean_squared_error: 7.8168e-06 - val_loss: 1.3471e-05 - val_mean_squared_error: 1.3471e-05
Epoch 96/100
7576/7576 [==============================] - 12s 2ms/step - loss: 8.8176

ResourceExhaustedError: OOM when allocating tensor with shape[1024,1024]
	 [[Node: time_distributed_12/random_uniform/RandomUniform = RandomUniform[T=DT_INT32, dtype=DT_FLOAT, seed=87654321, seed2=8724690, _device="/job:localhost/replica:0/task:0/device:GPU:0"](time_distributed_12/random_uniform/shape)]]

Caused by op 'time_distributed_12/random_uniform/RandomUniform', defined at:
  File "/usr/local/miniconda3/envs/dl/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/miniconda3/envs/dl/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/miniconda3/envs/dl/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/miniconda3/envs/dl/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/miniconda3/envs/dl/lib/python3.6/site-packages/ipykernel/kernelapp.py", line